In [1]:
# Import Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

# Get Search Information 
user_location = str(input("Enter Location: "))
user_search = str(input("Enter Business: "))

# Get URL For Top Hit in Search
search_url = "https://www.yelp.com/search?find_desc=" + user_search + "&find_loc=" + user_location
print(f"search url: {search_url}")
response = requests.get(search_url)
search_soup = bs(response.text,"lxml")

# Get Page Source For Business Page
raw_links = search_soup.find_all("a", class_="link-color--blue-dark__373c0__1mhJo")
all_links = []
[all_links.append(link.get("href")) for link in raw_links]
processed_links = []
[processed_links.append(link) for link in all_links if "/biz" in link]

business_page = "https://www.yelp.com" + processed_links[0]

response = requests.get(business_page)
business_soup = bs(response.text,"lxml")

business_name_raw = business_soup.find_all("h1",class_="biz-page-title")
business_name_list = []

[business_name_list.append(biz.text) for biz in business_name_raw]
business_name = " ".join(business_name_list)
print(f"business name: {business_name}")

comments = []
published_dates = []
ratings = []

for i in range(3):
    next_page = business_page + f"?start={i*20}"
    response = requests.get(next_page)
    business_soup = bs(response.text,"lxml")
    raw_comments = business_soup.find_all("p",itemprop="description")

    [comments.append(comment.text) for comment in raw_comments]

    raw_published_dates = business_soup.find_all("meta",itemprop="datePublished")

    [published_dates.append(comment.get("content")) for comment in raw_published_dates]

    raw_ratings = business_soup.find_all("meta",itemprop="ratingValue")
    
    business_rating = raw_ratings[0].get("content")
    
    raw_ratings = raw_ratings[1:len(raw_ratings)]
    
    [ratings.append(rating.get("content")) for rating in raw_ratings]

print(f"business overall rating: {business_rating}")

df = pd.DataFrame({"Published Date":published_dates,
                   "Comment": comments,
                   "Rating" : ratings})


Enter Location: dc
Enter Business: beefsteak
search url: https://www.yelp.com/search?find_desc=beefsteak&find_loc=dc
business name:  Beefsteak
business overall rating: 4.0


In [3]:
df.to_json(orient="records")

'[{"Published Date":"2019-02-09","Comment":"Downtown DC is pretty health centric, so it\'s no surprise that the food scene is awash with fast casual salad spots. Despite this, Beefsteak (named after the beefsteak tomato) manages to stand out. They boil their vegetables rather than dish them out raw, so if you\'re into that or you want a change of pace from your standard salad, you won\'t go wrong here. \\n\\nIngredients are also off the beaten path - examples include seaweed salad and poached egg. The offerings are fun and original. Most friends swear by the Frieda Kale (kale with black beans and spicy tomato sauce), while I\'m partial to the Kimchi Wa or Eden with seaweed salad.\\n\\nMost importantly, the beefsteak crew knows how to have a good time. In almost four years in the city, I have yet to find a team as friendly and offbeat as this one.","Rating":"5.0"},{"Published Date":"2018-12-27","Comment":"I had never been to this location and in fact had never heard of it until I was lo